In [3]:
#Import important packages

import os
import typing
from sklearn.gaussian_process.kernels import *
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
import matplotlib.pyplot as plt
from matplotlib import cm
import gpytorch
from matplotlib import rcParams
import torch
import pandas as pd
from sklearn.model_selection import train_test_split

In [4]:
# Set `EXTENDED_EVALUATION` to `True` in order to visualize your predictions.
EXTENDED_EVALUATION = True
EVALUATION_GRID_POINTS = 300  # Number of grid points used in extended evaluation
EVALUATION_GRID_POINTS_3D = 50  # Number of points displayed in 3D during evaluation


# Cost function constants
COST_W_UNDERPREDICT = 25.0
COST_W_NORMAL = 1.0
COST_W_OVERPREDICT = 10.0

In [5]:
class Model(gpytorch.models.ExactGP):
    def __init__(self, train_x, train_y, likelihood):
        super(Model, self).__init__(train_x, train_y, likelihood)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(
            gpytorch.kernels.RBFKernel(
                ard_num_dims=train_x.shape[1]
            )
        )

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)
    

def make_predictions(self, test_features: np.ndarray) -> typing.Tuple[np.ndarray, np.ndarray, np.ndarray]:
    """
    Predict the pollution concentration for a given set of locations.
    :param test_features: Locations as a 2d NumPy float array of shape (NUM_SAMPLES, 2)
    :return:
        Tuple of three 1d NumPy float arrays, each of shape (NUM_SAMPLES,),
        containing your predictions, the GP posterior mean, and the GP posterior stddev (in that order)
    """

    # TODO: Use your GP to estimate the posterior mean and stddev for each location here
    gp_mean = np.zeros(test_features.shape[0], dtype=float)
    gp_std = np.zeros(test_features.shape[0], dtype=float)

    return predictions, gp_mean, gp_std

def fitting_model(model, train_GT: np.ndarray,train_features: np.ndarray):
    """
    Fit your model on the given training data.
    :param train_features: Training features as a 2d NumPy float array of shape (NUM_SAMPLES, 2)
    :param train_GT: Training pollution concentrations as a 1d NumPy float array of shape (NUM_SAMPLES,)
    """

    # TODO: Fit your model here


In [6]:
def cost_function(ground_truth: np.ndarray, predictions: np.ndarray) -> float:
    """
    Calculates the cost of a set of predictions.

    :param ground_truth: Ground truth pollution levels as a 1d NumPy float array
    :param predictions: Predicted pollution levels as a 1d NumPy float array
    :return: Total cost of all predictions as a single float
    """
    assert ground_truth.ndim == 1 and predictions.ndim == 1 and ground_truth.shape == predictions.shape

    # Unweighted cost
    cost = (ground_truth - predictions) ** 2
    weights = np.ones_like(cost) * COST_W_NORMAL

    # Case i): underprediction
    mask_1 = predictions < ground_truth
    weights[mask_1] = COST_W_UNDERPREDICT

    # Case ii): significant overprediction
    mask_2 = (predictions >= 1.2*ground_truth)
    weights[mask_2] = COST_W_OVERPREDICT

    # Weigh the cost and return the average
    return np.mean(cost * weights)


In [7]:
def perform_extended_evaluation(model: Model, output_dir: str = '/results'):
    """
    Visualizes the predictions of a fitted model.
    :param model: Fitted model to be visualized
    :param output_dir: Directory in which the visualizations will be stored
    """
    print('Performing extended evaluation')
    fig = plt.figure(figsize=(30, 10))
    fig.suptitle('Extended visualization of task 1')

    # Visualize on a uniform grid over the entire coordinate system
    grid_lat, grid_lon = np.meshgrid(
        np.linspace(0, EVALUATION_GRID_POINTS - 1, num=EVALUATION_GRID_POINTS) / EVALUATION_GRID_POINTS,
        np.linspace(0, EVALUATION_GRID_POINTS - 1, num=EVALUATION_GRID_POINTS) / EVALUATION_GRID_POINTS,
    )
    visualization_xs = np.stack((grid_lon.flatten(), grid_lat.flatten()), axis=1)

    # Obtain predictions, means, and stddevs over the entire map
    predictions, gp_mean, gp_stddev = model.make_predictions(visualization_xs)
    predictions = np.reshape(predictions, (EVALUATION_GRID_POINTS, EVALUATION_GRID_POINTS))
    gp_mean = np.reshape(gp_mean, (EVALUATION_GRID_POINTS, EVALUATION_GRID_POINTS))
    gp_stddev = np.reshape(gp_stddev, (EVALUATION_GRID_POINTS, EVALUATION_GRID_POINTS))

    vmin, vmax = 0.0, 65.0
    vmax_stddev = 35.5

    # Plot the actual predictions
    ax_predictions = fig.add_subplot(1, 3, 1)
    predictions_plot = ax_predictions.imshow(predictions, vmin=vmin, vmax=vmax)
    ax_predictions.set_title('Predictions')
    fig.colorbar(predictions_plot)

    # Plot the raw GP predictions with their stddeviations
    ax_gp = fig.add_subplot(1, 3, 2, projection='3d')
    ax_gp.plot_surface(
        X=grid_lon,
        Y=grid_lat,
        Z=gp_mean,
        facecolors=cm.get_cmap()(gp_stddev / vmax_stddev),
        rcount=EVALUATION_GRID_POINTS_3D,
        ccount=EVALUATION_GRID_POINTS_3D,
        linewidth=0,
        antialiased=False
    )
    ax_gp.set_zlim(vmin, vmax)
    ax_gp.set_title('GP means, colors are GP stddev')

    # Plot the standard deviations
    ax_stddev = fig.add_subplot(1, 3, 3)
    stddev_plot = ax_stddev.imshow(gp_stddev, vmin=vmin, vmax=vmax_stddev)
    ax_stddev.set_title('GP estimated stddev')
    fig.colorbar(stddev_plot)

    # Save figure to pdf
    figure_path = os.path.join(output_dir, 'extended_evaluation.pdf')
    fig.savefig(figure_path)
    print(f'Saved extended evaluation to {figure_path}')

    plt.show()


In [ ]:
def main():
    # Load the training dateset and test features
    train_features = np.loadtxt('train_x.csv', delimiter=',', skiprows=1)
    train_GT = np.loadtxt('train_y.csv', delimiter=',', skiprows=1)
    test_features = np.loadtxt('test_x.csv', delimiter=',', skiprows=1)
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    
    #train_X,val_X,train_Y,val_Y = train_test_split(train_features, train_GT, 
    #                                               test_size=0.5, train_size=0.5, 
    #                                               random_state=0, shuffle=True)
    
    print(f"Is CUDA supported by this system? {torch.cuda.is_available()}")
    print(f"CUDA version: {torch.version.cuda}")
    
    train_X = torch.Tensor(train_features)
    train_Y = torch.Tensor(train_GT).squeeze()

    #train_X = train_X.cuda()
    #train_Y = train_Y.cuda()
    
    #val_X = torch.Tensor(val_X)
    #val_Y = torch.Tensor(val_Y).squeeze()
    
    print(f"train_X.shape: {train_X.shape}")
    print(f"train_y.shape: {train_Y.shape}")
    #print(f"val_X.shape: {val_X.shape}")
    #print(f"val_y.shape: {val_Y.shape}")
    
    # Fit the model
    print('Fitting model')
    
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = Model(train_X, train_Y, likelihood)
    
    #likelihood = likelihood.cuda()
    #model = model.cuda()
    
    model.train()
    likelihood.train()
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.1)
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)
    
    iterations = 100
    for i in range(iterations):
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(train_X)
        # Calc loss and backprop gradients
        loss = -mll(output, train_Y)
        loss.backward()
        print('Iteration %d/%d - Loss: %.3f' , (i, iterations, loss.item()))
        optimizer.step()

    #model = Model(train_features,train_GT,likelihood)
    #fitting_model(model,train_GT,train_features)

    # Predict on the test features
    #print('Predicting on test features')
    #predictions, gp_mean, gp_std = model.make_predictions(test_features)
    
    #print(predictions)
    
    #cost_function(train_GT, predictions)

    #if EXTENDED_EVALUATION:
    #    perform_extended_evaluation(model, output_dir='.')


if __name__ == "__main__":
    main()


Is CUDA supported by this system? False
CUDA version: None
train_X.shape: torch.Size([15189, 2])
train_y.shape: torch.Size([15189])
Fitting model
Iteration %d/%d - Loss: %.3f (0, 100, 171.46826171875)
Iteration %d/%d - Loss: %.3f (1, 100, 157.45913696289062)
Iteration %d/%d - Loss: %.3f (2, 100, 144.59556579589844)
Iteration %d/%d - Loss: %.3f (3, 100, 132.86122131347656)
Iteration %d/%d - Loss: %.3f (4, 100, 122.26272583007812)
Iteration %d/%d - Loss: %.3f (5, 100, 112.78837585449219)
Iteration %d/%d - Loss: %.3f (6, 100, 104.35353088378906)
Iteration %d/%d - Loss: %.3f (7, 100, 96.80535888671875)
Iteration %d/%d - Loss: %.3f (8, 100, 90.01158905029297)
Iteration %d/%d - Loss: %.3f (9, 100, 83.91144561767578)
Iteration %d/%d - Loss: %.3f (10, 100, 78.43408203125)
Iteration %d/%d - Loss: %.3f (11, 100, 73.41201782226562)
Iteration %d/%d - Loss: %.3f (12, 100, 68.6243667602539)
Iteration %d/%d - Loss: %.3f (13, 100, 63.91855239868164)
Iteration %d/%d - Loss: %.3f (14, 100, 59.2628326416